In [1]:
import pandas as pd
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
from future.utils import iteritems
from collections import Counter
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import pymysql
from gensim.models import word2vec
from tqdm.notebook import tqdm

import sys
if sys.version_info <= (2,7):
    reload(sys)
    sys.setdefaultencoding('utf-8')
import konlpy
from konlpy.tag import Kkma, Okt, Hannanum

In [2]:
article_data = pd.read_csv('mixver10.txt', encoding='utf-8', header= None)
documents = [' '.join(i[0].split(' ')[1:]) for i in article_data.values]

as_one = ''
for document in documents:
    as_one = as_one + ' ' + document
words = as_one.split()

counts = Counter(words)

vocab = sorted(counts, key=counts.get, reverse=True)

word2idx = {word.encode("utf8").decode("utf8"): ii for ii, word in enumerate(vocab,1)}

idx2word = {ii: word for ii, word in enumerate(vocab)}

V = len(word2idx)
N = len(documents)

tf = CountVectorizer()

tf.fit_transform(documents)

tf.fit_transform(documents)[0:1].toarray()

tfidf = TfidfVectorizer(max_features = 1800, max_df=1, min_df=0)

#generate tf-idf term-document matrix
A_tfidf_sp = tfidf.fit_transform(documents)  #size D x V

tfidf_dict = tfidf.get_feature_names()

In [3]:
kkma = Kkma()
okt = Okt()
hannanum = Hannanum()
print('konlpy version = %s' % konlpy.__version__)

konlpy version = 0.5.2


In [4]:
conn = pymysql.connect(host='127.0.0.1', user='root', password='wdta2181',db='test', charset='utf8')
curs = conn.cursor()
# input : category, title_keyword(only noun)
# 토트백 -> 토트 백 (가방 종류 칭할 때 '백'은 띄워서 써주세요라고 명시하기)

project_category ='출판'
#title로 받은 경우
project_title = '아이와 어른이 함께 보는 출근하는 아이소파로 간 아이'

selected_word = list(set(okt.nouns(project_title)))


model = word2vec.Word2Vec.load("NaverMovie30.model")
#keyword로 받은 경우
#selected_word = ['그림책', '동화']

print(selected_word)

t_cnt =0
w_cnt = 0

similar_word_list=[]
for i in selected_word:
    try:
        similar_word=model.wv.most_similar(positive=[i],topn=2)
    except:
        continue
    for j in similar_word:

        if len(j[0]) == 1:
            print()
            continue
        else:
            similar_word_list.append(j[0])
if len(selected_word) < 5:
    similar_word_list = similar_word_list + selected_word
            
similar_word_set = set(similar_word_list)    
print("similar_word_set",end='')
print(similar_word_set)
similar_word_set
res_list=[]

for i in similar_word_set:
    sql = 'select pagename,trim(title) from test.crawl where category="%s" and title like "%%%s%%" and achieve>=90;'%(project_category,i)
    curs.execute(sql)
    pagename = curs.fetchall()

    length = len(pagename)
    if length >3:
        length = 3

    for k in range(0,length):
        res_list.append((pagename[k][0], pagename[k][1]))

res_set = set(res_list)
for k in res_set:
    if k[0] == 'tumblbug':
        print("T "+k[1])
        t_cnt+=1
    elif k[0] == 'wadiz':
        print("W "+k[1])
        w_cnt+=1
    else:
        continue
    print()
print()
print("T W")
print(t_cnt, w_cnt)
if t_cnt > w_cnt:
    print("tumblbug")
elif t_cnt == w_cnt:
    print("same")
else:
    print("wadiz")
conn.close()

['어른', '아이', '소파', '간', '출근']
similar_word_set{'일력', '싱크', '드로잉', '독립', '하늘', '식탁', '현상', '공기청정기', '마련', '감옥'}
T 노정이 쓰고 드로잉메리가 그린 소설 달콤한 밤 되세요

T 빛나는 인생후반을 위한 추억 드로잉 “별들의 고향”

T 조선 밤하늘 별자리를 담다 조선천문도감

T 여성에게 국가는 없다 한남·여혐민국의 20대여성 현상

T 스물아홉 결혼 말고 독립일기 당신도 혼자 살고 있나요

T 취향따라 떠나는 제주 드로잉투어컬러링북 제주

T 승무원을 꿈꾸는 그대에게 아름다운 하늘을 선물합니다

T PDF 자취방 구하기 초보를 위한 내집마련 가이드북

T 인터뷰놀이X인터뷰집 가을 하늘 말갛게 뜬

T 할 수 있다!독립출판제작자를 위한 대형서점 유통 가이드

W 2차 펀딩 베스트셀러에 시들해진 그대에게_독립출판물 정기구독 서비스


T W
10 1
tumblbug


# keyword

In [25]:
conn = pymysql.connect(host='127.0.0.1', user='root', password='wdta2181',db='test', charset='utf8')
curs = conn.cursor()
# input : category, title_keyword(only noun)
# 토트백 -> 토트 백 (가방 종류 칭할 때 '백'은 띄워서 써주세요라고 명시하기)

project_category ='온라인게임'


model = word2vec.Word2Vec.load("NaverMovie30.model")
#keyword로 받은 경우
selected_word = ['성착취', '사회', '게임']

print(selected_word)

t_cnt =0
w_cnt = 0

similar_word_list=[]
for i in selected_word:
    try:
        similar_word=model.wv.most_similar(positive=[i],topn=2)
    except:
        continue
    for j in similar_word:

        if len(j[0]) == 1:
            print()
            continue
        else:
            similar_word_list.append(j[0])
if len(selected_word) < 5:
    similar_word_list = similar_word_list + selected_word
            
similar_word_set = set(similar_word_list)    
print("similar_word_set",end='')
print(similar_word_set)
similar_word_set
res_list=[]

for i in similar_word_set:
    sql = 'select pagename,trim(title) from test.crawl where category="%s" and title like "%%%s%%" and achieve>=90;'%(project_category,i)
    curs.execute(sql)
    pagename = curs.fetchall()

    length = len(pagename)
    if length >3:
        length = 3

    for k in range(0,length):
        res_list.append((pagename[k][0], pagename[k][1]))

res_set = set(res_list)
for k in res_set:
    if k[0] == 'tumblbug':
        print("T "+k[1])
        t_cnt+=1
    elif k[0] == 'wadiz':
        print("W "+k[1])
        w_cnt+=1
    else:
        continue
    print()
print()
print("T W")
print(t_cnt, w_cnt)
if t_cnt > w_cnt:
    print("결과 : tumblbug")
elif t_cnt == w_cnt:
    print("same")
else:
    print("결과 : wadiz")
conn.close()

['성착취', '사회', '게임']
similar_word_set{'여름', '성착취', '보약', '사회', '손가락', '제작자', '게임'}
T 그리스 신화 기반 여성향 동인게임 제우스의 대리인 1부

T 여름날의 감동을 담은 비주얼노벨 여름의 끝에 피는 꽃

T 내 손에 펼쳐지는 로맨스 바른연애 길잡이시뮬레이션 게임

W 6가지 게임모드를 지원하는 단 하나의 게이밍 퍼포먼스 체어 지니디아


T W
3 1
결과 : tumblbug


# title

In [24]:

conn = pymysql.connect(host='127.0.0.1', user='root', password='wdta2181',db='test', charset='utf8')
curs = conn.cursor()
# input : category, title_keyword(only noun)
# 토트백 -> 토트 백 (가방 종류 칭할 때 '백'은 띄워서 써주세요라고 명시하기)

project_category ='테크'
#title로 받은 경우
project_title = '휴대폰 만능거치대한국상륙! 일상사무실여행캠핑 활용성 갑 of 갑!'

tokenized_project_title = set(okt.nouns(project_title))
selected_word = []

for i in tokenized_project_title:
    if i in tfidf_dict:
        selected_word.append(i)
        continue

model = word2vec.Word2Vec.load("NaverMovie30.model")


print(selected_word)

t_cnt =0
w_cnt = 0

similar_word_list=[]
for i in selected_word:
    try:
        similar_word=model.wv.most_similar(positive=[i],topn=2)
    except:
        continue
    for j in similar_word:

        if len(j[0]) == 1:
            print()
            continue
        else:
            similar_word_list.append(j[0])
if len(selected_word) < 5:
    similar_word_list = similar_word_list + selected_word
            
similar_word_set = set(similar_word_list)    
print("similar_word_set",end='')
print(similar_word_set)
similar_word_set
res_list=[]

for i in similar_word_set:
    sql = 'select pagename,trim(title) from test.crawl where category="%s" and title like "%%%s%%" and achieve>=90;'%(project_category,i)
    curs.execute(sql)
    pagename = curs.fetchall()

    length = len(pagename)
    if length >3:
        length = 3

    for k in range(0,length):
        res_list.append((pagename[k][0], pagename[k][1]))

res_set = set(res_list)
for k in res_set:
    if k[0] == 'tumblbug':
        print("T "+k[1])
        t_cnt+=1
    elif k[0] == 'wadiz':
        print("W "+k[1])
        w_cnt+=1
    else:
        continue
    print()
print()
print("T W")
print(t_cnt, w_cnt)
if t_cnt > w_cnt:
    print("결과 : tumblbug")
elif t_cnt == w_cnt:
    print("same")
else:
    print("결과 : wadiz")
conn.close()

['휴대폰', '일상', '캠핑', '활용', '거치', '만능', '여행']

similar_word_set{'테이블', '다이어리', '마우스', '여행지', '쥬얼', '카프', '자꾸', '기억', '근적외선', '포토', '탄생석', '대세', '아이패드'}
W 국내독점 노마드 아이패드 프로 천연 가죽 케이스

W 당신의 아이패드프로를 맥북으로 도코 올인원 키보드 케이스

W 5차 앵콜 나만의 이니셜을 가진 특별한 무선충전 폴더블 마우스패드

W 글로벌 1억펀딩 뻐근한 내 몸 근적외선 빛으로 전신 찜질!

W 여행의 기억을 세계지도에 차곡차곡 별빛 지도 무드등 사이너


T W
0 5
결과 : wadiz


In [27]:
conn = pymysql.connect(host='127.0.0.1', user='root', password='wdta2181',db='test', charset='utf8')
curs = conn.cursor()
# input : category, title_keyword(only noun)
# 토트백 -> 토트 백 (가방 종류 칭할 때 '백'은 띄워서 써주세요라고 명시하기)

project_category ='테크'


model = word2vec.Word2Vec.load("NaverMovie30.model")
#keyword로 받은 경우
selected_word = ['휴대폰', '거치대']

print(selected_word)

t_cnt =0
w_cnt = 0

similar_word_list=[]
for i in selected_word:
    try:
        similar_word=model.wv.most_similar(positive=[i],topn=2)
    except:
        continue
    for j in similar_word:

        if len(j[0]) == 1:
            print()
            continue
        else:
            similar_word_list.append(j[0])
if len(selected_word) < 5:
    similar_word_list = similar_word_list + selected_word
            
similar_word_set = set(similar_word_list)    
print("similar_word_set",end='')
print(similar_word_set)
similar_word_set
res_list=[]

for i in similar_word_set:
    sql = 'select pagename,trim(title) from test.crawl where category="%s" and title like "%%%s%%" and achieve>=90;'%(project_category,i)
    curs.execute(sql)
    pagename = curs.fetchall()

    length = len(pagename)
    if length >3:
        length = 3

    for k in range(0,length):
        res_list.append((pagename[k][0], pagename[k][1]))

res_set = set(res_list)
for k in res_set:
    if k[0] == 'tumblbug':
        print("T "+k[1])
        t_cnt+=1
    elif k[0] == 'wadiz':
        print("W "+k[1])
        w_cnt+=1
    else:
        continue
    print()
print()
print("T W")
print(t_cnt, w_cnt)
if t_cnt > w_cnt:
    print("결과 : tumblbug")
elif t_cnt == w_cnt:
    print("same")
else:
    print("결과 : wadiz")
conn.close()

['휴대폰', '거치대']
similar_word_set{'근적외선', '카프', '휴대폰', '거치대'}
W 걸리버거치대스마트폰과 테블릿을 같이 쓸수 있는 차량용 휴대폰 거치대

W 2029달성 앵콜펀딩 무선충전과 스피커까지 담은 모빌리티 거치대

W 글로벌11억펀딩MOFT 노트북태블릿 파우치와 거치대를 하나로!

W 글로벌 1억펀딩 뻐근한 내 몸 근적외선 빛으로 전신 찜질!

W 업그레이드 앵콜 다시한번 가격파괴 차량용 무선충전 거치대 핸디그랩


T W
0 5
결과 : wadiz


In [28]:

conn = pymysql.connect(host='127.0.0.1', user='root', password='wdta2181',db='test', charset='utf8')
curs = conn.cursor()
# input : category, title_keyword(only noun)
# 토트백 -> 토트 백 (가방 종류 칭할 때 '백'은 띄워서 써주세요라고 명시하기)

project_category ='여행·레저'
#title로 받은 경우
project_title = '대를 이어 사용 가능할만큼 터프한 강철 루핀 화로대'

tokenized_project_title = set(okt.nouns(project_title))
selected_word = []

for i in tokenized_project_title:
    if i in tfidf_dict:
        selected_word.append(i)
        continue

model = word2vec.Word2Vec.load("NaverMovie30.model")


print(selected_word)

t_cnt =0
w_cnt = 0

similar_word_list=[]
for i in selected_word:
    try:
        similar_word=model.wv.most_similar(positive=[i],topn=2)
    except:
        continue
    for j in similar_word:

        if len(j[0]) == 1:
            print()
            continue
        else:
            similar_word_list.append(j[0])
if len(selected_word) < 5:
    similar_word_list = similar_word_list + selected_word
            
similar_word_set = set(similar_word_list)    
print("similar_word_set",end='')
print(similar_word_set)
similar_word_set
res_list=[]

for i in similar_word_set:
    sql = 'select pagename,trim(title) from test.crawl where category="%s" and title like "%%%s%%" and achieve>=90;'%(project_category,i)
    curs.execute(sql)
    pagename = curs.fetchall()

    length = len(pagename)
    if length >3:
        length = 3

    for k in range(0,length):
        res_list.append((pagename[k][0], pagename[k][1]))

res_set = set(res_list)
for k in res_set:
    if k[0] == 'tumblbug':
        print("T "+k[1])
        t_cnt+=1
    elif k[0] == 'wadiz':
        print("W "+k[1])
        w_cnt+=1
    else:
        continue
    print()
print()
print("T W")
print(t_cnt, w_cnt)
if t_cnt > w_cnt:
    print("결과 : tumblbug")
elif t_cnt == w_cnt:
    print("same")
else:
    print("결과 : wadiz")
conn.close()

['사용', '화로']
similar_word_set{'사용', '필수품', '화로', '다큐멘터리', '로서', '컴백'}
W 국내 제작 사용해보면 알아요 베게를 열면 담요가??

W 화로 유목민들 주목! 캠핑의 고민을 해결할 마지막 화로 캠프에디션!

W 특허받은 원액션 원목 화로테이블. 실용성과 감성을 모두 담은캠핑 테이블

W 캠핑의 꽃!아노다이징 코팅이 만들어낸 웰메이드 화로 봉화로


T W
0 4
결과 : wadiz


In [30]:
conn = pymysql.connect(host='127.0.0.1', user='root', password='wdta2181',db='test', charset='utf8')
curs = conn.cursor()
# input : category, title_keyword(only noun)
# 토트백 -> 토트 백 (가방 종류 칭할 때 '백'은 띄워서 써주세요라고 명시하기)

project_category ='여행·레저'


model = word2vec.Word2Vec.load("NaverMovie30.model")
#keyword로 받은 경우
selected_word = ['화로', '캠프']

print(selected_word)

t_cnt =0
w_cnt = 0

similar_word_list=[]
for i in selected_word:
    try:
        similar_word=model.wv.most_similar(positive=[i],topn=2)
    except:
        continue
    for j in similar_word:

        if len(j[0]) == 1:
            print()
            continue
        else:
            similar_word_list.append(j[0])
if len(selected_word) < 5:
    similar_word_list = similar_word_list + selected_word
            
similar_word_set = set(similar_word_list)    
print("similar_word_set",end='')
print(similar_word_set)
similar_word_set
res_list=[]

for i in similar_word_set:
    sql = 'select pagename,trim(title) from test.crawl where category="%s" and title like "%%%s%%" and achieve>=90;'%(project_category,i)
    curs.execute(sql)
    pagename = curs.fetchall()

    length = len(pagename)
    if length >3:
        length = 3

    for k in range(0,length):
        res_list.append((pagename[k][0], pagename[k][1]))

res_set = set(res_list)
for k in res_set:
    if k[0] == 'tumblbug':
        print("T "+k[1])
        t_cnt+=1
    elif k[0] == 'wadiz':
        print("W "+k[1])
        w_cnt+=1
    else:
        continue
    print()
print()
print("T W")
print(t_cnt, w_cnt)
if t_cnt > w_cnt:
    print("결과 : tumblbug")
elif t_cnt == w_cnt:
    print("same")
else:
    print("결과 : wadiz")
conn.close()

['화로', '캠프']
similar_word_set{'다비드', '화로', '다큐멘터리', '캠프', '안성맞춤', '로서'}
W 16ft가 제안하는 캠프기어 과거와 현재를 아우르는 텐트 루트베이스

W 특허받은 원액션 원목 화로테이블. 실용성과 감성을 모두 담은캠핑 테이블

W 캠핑의 꽃!아노다이징 코팅이 만들어낸 웰메이드 화로 봉화로

W 화로 유목민들 주목! 캠핑의 고민을 해결할 마지막 화로 캠프에디션!

W 앵콜 당신을 위한 낭만 캠핑 감성 면텐트 로그캐빈 그라비티캠프


T W
0 5
결과 : wadiz


In [35]:

conn = pymysql.connect(host='127.0.0.1', user='root', password='wdta2181',db='test', charset='utf8')
curs = conn.cursor()
# input : category, title_keyword(only noun)
# 토트백 -> 토트 백 (가방 종류 칭할 때 '백'은 띄워서 써주세요라고 명시하기)

project_category ='온라인게임'
#title로 받은 경우
project_title = '성착취 사회 문제를 다룬 시리어스 게임 Slanted Structure'

tokenized_project_title = set(okt.nouns(project_title))
selected_word = []

for i in tokenized_project_title:
    if i in tfidf_dict:
        selected_word.append(i)
        continue

model = word2vec.Word2Vec.load("NaverMovie30.model")


print(selected_word)

t_cnt =0
w_cnt = 0

similar_word_list=[]
for i in selected_word:
    try:
        similar_word=model.wv.most_similar(positive=[i],topn=2)
    except:
        continue
    for j in similar_word:

        if len(j[0]) == 1:
            print()
            continue
        else:
            similar_word_list.append(j[0])
if len(selected_word) < 5:
    similar_word_list = similar_word_list + selected_word
            
similar_word_set = set(similar_word_list)    
print("similar_word_set",end='')
print(similar_word_set)
similar_word_set
res_list=[]

for i in similar_word_set:
    sql = 'select pagename,trim(title) from test.crawl where category="%s" and title like "%%%s%%" and achieve>=90;'%(project_category,i)
    curs.execute(sql)
    pagename = curs.fetchall()

    length = len(pagename)
    if length >3:
        length = 3

    for k in range(0,length):
        res_list.append((pagename[k][0], pagename[k][1]))

res_set = set(res_list)
for k in res_set:
    if k[0] == 'tumblbug':
        print("T "+k[1])
        t_cnt+=1
    elif k[0] == 'wadiz':
        print("W "+k[1])
        w_cnt+=1
    else:
        continue
    print()
print()
print("T W")
print(t_cnt, w_cnt)
if t_cnt > w_cnt:
    print("결과 : tumblbug")
elif t_cnt == w_cnt:
    print("same")
else:
    print("결과 : wadiz")
conn.close()

['문제', '사회', '게임']
similar_word_set{'감자', '여름', '농구', '보약', '문제', '사회', '손가락', '제작자', '게임'}
T 그리스 신화 기반 여성향 동인게임 제우스의 대리인 1부

T 여름날의 감동을 담은 비주얼노벨 여름의 끝에 피는 꽃

T 내 손에 펼쳐지는 로맨스 바른연애 길잡이시뮬레이션 게임

W 6가지 게임모드를 지원하는 단 하나의 게이밍 퍼포먼스 체어 지니디아


T W
3 1
결과 : tumblbug


In [34]:
conn = pymysql.connect(host='127.0.0.1', user='root', password='wdta2181',db='test', charset='utf8')
curs = conn.cursor()
# input : category, title_keyword(only noun)
# 토트백 -> 토트 백 (가방 종류 칭할 때 '백'은 띄워서 써주세요라고 명시하기)

project_category ='예술'


model = word2vec.Word2Vec.load("NaverMovie30.model")
#keyword로 받은 경우
selected_word = ['강아지', '댕댕', '에코', '백']

print(selected_word)

t_cnt =0
w_cnt = 0

similar_word_list=[]
for i in selected_word:
    try:
        similar_word=model.wv.most_similar(positive=[i],topn=2)
    except:
        continue
    for j in similar_word:

        if len(j[0]) == 1:
            print()
            continue
        else:
            similar_word_list.append(j[0])
if len(selected_word) < 5:
    similar_word_list = similar_word_list + selected_word
            
similar_word_set = set(similar_word_list)    
print("similar_word_set",end='')
print(similar_word_set)
similar_word_set
res_list=[]

for i in similar_word_set:
    sql = 'select pagename,trim(title) from test.crawl where category="%s" and title like "%%%s%%" and achieve>=90;'%(project_category,i)
    curs.execute(sql)
    pagename = curs.fetchall()

    length = len(pagename)
    if length >3:
        length = 3

    for k in range(0,length):
        res_list.append((pagename[k][0], pagename[k][1]))

res_set = set(res_list)
for k in res_set:
    if k[0] == 'tumblbug':
        print("T "+k[1])
        t_cnt+=1
    elif k[0] == 'wadiz':
        print("W "+k[1])
        w_cnt+=1
    else:
        continue
    print()
print()
print("T W")
print(t_cnt, w_cnt)
if t_cnt > w_cnt:
    print("결과 : tumblbug")
elif t_cnt == w_cnt:
    print("same")
else:
    print("결과 : wadiz")
conn.close()

['강아지', '댕댕', '에코', '백']

similar_word_set{'넥타이', '가방', '수납', '부적', '자녀', '강아지', '백', '크로스', '댕댕', '에코', '노트북'}
T 소소한 상상을 그려요 일러스트 파우치에코백 3set

T 노윤과 함께 나누는 데일리 백

T 생각지도 못한 질문 고백 안부

T 일상 속 방 한 켠에서 만나는 소양강 페브릭포스터 by백하


T W
4 0
결과 : tumblbug
